In [1]:
from operator import index
import pandas as pd
import os
import math
import seaborn as sns
import matplotlib.pyplot as plt

In this project there are four main goals:
1. to count how many lines are in a drawing
2. to calculate the length of each line and the mean pressure when computing each line
3. to check how many hand lifts were in each drawing
4. to check if the shape is open or closed (bonus)

### Attempts to goal no' 1 - count how many lines are in a drawing:

#### 2. calculating the length of each line and the mean pressure when computing each line:

In [ ]:
def find_lines_length_and_pressure_by_index(df, indexes):
    #df is the dataframe, indexes is a list of the start/ end points of each line in the drawing

#### 3. checking how many hand lifts were in each drawing

In [ ]:
def find_hand_off_by_pressure(df):
    ave_pressure = df[' Pressure'].mean()
    min_pressure = df[' Pressure'].min()
    max_pressure = df[' Pressure'].max()
    smallest_values = df[df[' Pressure'] < (min_pressure + max_pressure)/2].sort_index()
    smallest_values['idx'] = smallest_values.index
    smallest_values['idx_shift'] = smallest_values.shift()['idx']
    smalest_values = smalest_values.drop([0])
    num_of_hand_lifts = (smallest_values['idx']-smallest_values['idx_shift']).apply(lambda x: 0 if x==1.0 else 1).sum() + 1
    print("The number of hand lifts computed by pressure is: ", num_of_hand_lifts)